In [1]:
import numpy as np
import pandas as pd
import plotnine as gg

from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

In [175]:
df = pd.read_csv('../data/engineered_factset_campaign_data.csv', parse_dates=[
    '18_months_pre_announcement_date',
    '1_year_pre_announcement_date',
    '6_months_pre_announcement_date',
    '90_days_pre_announcement_date',
    'campaign_announcement_date',
    '6_months_post_announcement_date',
    '1_year_post_announcement_date',
    '18_months_post_announcement_date'
])

df_model_data = (
    df
    .assign(objective=lambda df: df.campaign_objective_group)
    .assign(result=lambda df: df.proxy_campaign_winner_or_result)
    .assign(post_return=lambda df: df['1_year_post_date_total_return_category'])
)

In [177]:
df_model_data.groupby(['objective'])['1_year_post_date_total_return'].mean()

objective
Board Add          0.056824
Board Remove      -0.078587
ESG                0.039628
Merger             0.024228
Value Long         0.059467
Value Short       -0.018855
Vote Dissident     0.083221
Vote Management    0.069163
Name: 1_year_post_date_total_return, dtype: float64

In [182]:
model = BayesianModel([
    ('objective', 'post_return')
])

model.fit(
    df_model_data,
    estimator=BayesianEstimator,
    complete_samples_only=False
)

for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of objective:
+----------------------------+-----------+
| objective(Board Add)       | 0.351157  |
+----------------------------+-----------+
| objective(Board Remove)    | 0.0176232 |
+----------------------------+-----------+
| objective(ESG)             | 0.032527  |
+----------------------------+-----------+
| objective(Merger)          | 0.13445   |
+----------------------------+-----------+
| objective(Value Long)      | 0.249835  |
+----------------------------+-----------+
| objective(Value Short)     | 0.0211088 |
+----------------------------+-----------+
| objective(Vote Dissident)  | 0.180844  |
+----------------------------+-----------+
| objective(Vote Management) | 0.0124549 |
+----------------------------+-----------+
CPD of post_return:
+--------------------------+----------------------+-------------------------+--------------------+---------------------+-----------------------+------------------------+---------------------------+----------------------------+
| ob

In [183]:
model = BayesianModel([
    ('objective', 'result'),
    ('result', 'post_return')
])

model.fit(
    df_model_data,
    estimator=BayesianEstimator,
    complete_samples_only=False
)

for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of objective:
+----------------------------+-----------+
| objective(Board Add)       | 0.351157  |
+----------------------------+-----------+
| objective(Board Remove)    | 0.0176232 |
+----------------------------+-----------+
| objective(ESG)             | 0.032527  |
+----------------------------+-----------+
| objective(Merger)          | 0.13445   |
+----------------------------+-----------+
| objective(Value Long)      | 0.249835  |
+----------------------------+-----------+
| objective(Value Short)     | 0.0211088 |
+----------------------------+-----------+
| objective(Vote Dissident)  | 0.180844  |
+----------------------------+-----------+
| objective(Vote Management) | 0.0124549 |
+----------------------------+-----------+
CPD of result:
+----------------------------------+----------------------+-------------------------+----------------------+-----------------------+-----------------------+------------------------+---------------------------+---------------------------

# Appendix

In [35]:
df['1_year_post_date_total_return'].unique()

array([ 5.3903700e-04,            nan,  1.2344503e-03, ...,
       -1.0526312e-03,  4.7933817e-04, -6.3291190e-05])

In [24]:
df['1_year_post_date_total_return'].describe()

count    7521.000000
mean        0.000805
std         0.015886
min        -0.010000
25%        -0.002567
50%         0.000154
75%         0.002464
max         1.240000
Name: 1_year_post_date_total_return, dtype: float64

In [49]:
df.groupby(['campaign_objective_primary', 'proxy_campaign_winner_or_result']).campaign_id.count()

campaign_objective_primary                                proxy_campaign_winner_or_result
Board Control                                             Dissident                          140
                                                          Management                         151
                                                          Pending                             17
                                                          Settled/Concessions Made           226
                                                          Split                               14
                                                          Withdrawn                          190
Board Representation                                      Dissident                          163
                                                          Management                         287
                                                          Pending                             20
                                     

In [47]:
df.groupby(['campaign_objective_primary']).campaign_id.count().to_frame()

,campaign_id
campaign_objective_primary,
13D Filer - No Publicly Disclosed Activism,1256
Board Control,790
Board Representation,2131
Enhance Corporate Governance,270
Hostile/Unsolicited Acquisition,556
Maximize Shareholder Value,2078
Public Short Position/Bear Raid,175
"Remove Director(s), No Dissident Nominee to Fill Vacancy",97
Remove Officer(s),49


In [48]:
df.groupby(['proxy_campaign_winner_or_result']).campaign_id.count().to_frame()

,campaign_id
proxy_campaign_winner_or_result,
Dissident,358
Management,537
Pending,45
Settled/Concessions Made,729
Split,58
Withdrawn,476


In [4]:
df.tail(1)

,campaign_id,campaign_announcement_date,campaign_title,campaign_objective_primary,value_demand,governance_demand,activist_campaign_tactic,total_number_of_board_seats,number_of_board_seats_sought,short_or_majority_or_full_slate,...,used_proxy_access_nomination_tactic,used_publicly_disclosed_letter_to_board_or_management_tactic,used_take_action_by_written_consent_tactic,used_tender_offer_launched_tactic,used_tender_offer_stake_only_tactic,used_threaten_proxy_fight_tactic,used_unsolicited_offer_tactic,used_withhold_vote_for_directors_tactic,lagged_campaign_announcement_date,past_return_successes
9570,1073549609C,2018-12-06,"Del Frisco's Restaurant Group, Inc. / Engaged ...",Maximize Shareholder Value,Review Strategic Alternatives,Add Independent Directors,Publicly Disclosed Letter to Board/Management,0,0,NaN,...,0,1,0,0,0,0,0,0,2018-08-15,9.0


In [8]:
[print(x) for x in df.columns.tolist()];

campaign_id
campaign_announcement_date
campaign_title
campaign_objective_primary
value_demand
governance_demand
activist_campaign_tactic
total_number_of_board_seats
number_of_board_seats_sought
short_or_majority_or_full_slate
proxy_proposal
glass_lewis_support
iss_support
activist_id
activist_name
activist_group
first_trade_date
last_trade_date
ownership_pecent_on_announcements
company_id
company_name
sector
price_at_announcement
ltm_eps_at_announcement
earnings_yield_at_announcement
current_entity_status
current_entity_detail
public_before_or_after_campaign_announcement
poison_pill_in_force_prior_to_announcement
poison_pill_adopted_in_response_to_campaign
18_months_pre_announcement_date
1_year_pre_announcement_date
6_months_pre_announcement_date
90_days_pre_announcement_date
18_months_pre_date_total_return
1_year_pre_date_total_return
6_months_pre_date_total_return
90_days_pre_date_total_return
number_of_board_seats_gained
proxy_campaign_winner_or_result
activist_campaign_results
6_mo